In [1]:
import cv2
import os
import glob
import time
from ultralytics import YOLO
import warnings
!pwd

/Users/jeongmungyeong/Desktop/Project/VSCode/yolo8-RPS


In [2]:
# 객체 탐지 및 분류 모델 불러오기
try:
    detect_model = YOLO('./yolov8n.pt')  # 객체 탐지 모델
    classify_model = YOLO('./models/yolov8_clsRPS.pt')  # 분류 모델
except Exception as e:
    print(f"모델 로딩 중 오류 발생: {e}")
    exit()

# 객체 사진을 저장할 폴더 설정
save_dir = "class"  # 객체 사진을 저장할 폴더 이름
os.makedirs(save_dir, exist_ok=True)

# 분류된 객체의 이미지를 저장할 폴더 설정 (원하는 경로로 변경 가능)
classified_save_dir = "./class"
os.makedirs(classified_save_dir, exist_ok=True)

# 웹캠 설정 (0번 카메라 사용)
cap = cv2.VideoCapture(1)

# 웹캠이 열리지 않는 경우 종료
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

2024-09-11 22:05:25.802 python[48593:4105024] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [3]:
# 바운딩 박스를 확장하는 비율
EXPAND_RATIO = 0.0  # 20% 더 넓게 확장

# 폴더 내의 모든 파일 삭제 함수
def clear_folder(folder_path):
    try:
        files = glob.glob(f"{folder_path}/*")
        for f in files:
            os.remove(f)
    except Exception as e:
        print(f"폴더 정리 중 오류 발생: {e}")

In [4]:
# 분류 작업 수행 함수 (이미지 저장 포함)
def classify_image(image):
    try:
        # 이미지를 분류 모델에 직접 전달하여 예측 수행
        results = classify_model.predict(source=image)
        # 분류 결과 얻기
        if results and len(results) > 0:
            # 각 클래스의 확률을 가져와서 텍스트로 표시
            top1_probs = results[0].probs.data  # 모든 클래스 확률 가져오기
            class_names = results[0].names  # 클래스 이름들
            
            # 확률과 클래스 이름 매칭
            prob_text = ', '.join([f"{class_names[i]}: {prob:.2f}" for i, prob in enumerate(top1_probs)])
            
            # 분류된 결과를 저장 (예시)
            save_path = os.path.join(classified_save_dir, f"classification_{int(time.time() * 1000)}.jpg")
            cv2.imwrite(save_path, image)  # 분류된 이미지를 저장
            
            return prob_text
    except Exception as e:
        print(f"예기치 않은 분류 오류: {e}")
    return None

In [5]:
# 객체 탐지 및 분류 함수
def run_detection_and_classification():
    clear_folder(save_dir)
    
    # 명시적으로 창 생성
    cv2.namedWindow("Detection and Classification", cv2.WINDOW_NORMAL)
    while True:
        try:
            # 시작 시 class 폴더의 모든 파일 삭제
            # clear_folder(save_dir)

            ret, frame = cap.read()
            if not ret:
                print("카메라에서 프레임을 가져올 수 없습니다.")
                break  # 문제 발생 시 루프 종료

            # 객체 탐지 (저장 비활성화)
            try:
                detection_results = detect_model.predict(source=frame, save=False, save_crop=False, show=False)
            except Exception as e:
                print(f"객체 탐지 중 오류 발생: {e}")
                continue

            # 객체 탐지 시 분류 수행
            for result in detection_results:
                for box in result.boxes:
                    # 탐지된 객체의 클래스 이름 가져오기
                    class_id = int(box.cls[0])
                    class_name = result.names[class_id]
                    
                    # 'person' 클래스인 경우에만 분류 수행
                    if class_name == 'person':
                        # 바운딩 박스 확장
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        width = x2 - x1
                        height = y2 - y1

                        # 확장된 좌표 계산
                        expand_x = int(width * EXPAND_RATIO)
                        expand_y = int(height * EXPAND_RATIO)

                        # 확장된 좌표가 이미지 범위를 넘어가지 않도록 조정
                        x1 = max(0, x1 - expand_x)
                        y1 = max(0, y1 - expand_y)
                        x2 = min(frame.shape[1], x2 + expand_x)
                        y2 = min(frame.shape[0], y2 + expand_y)

                        try:
                            # 확장된 영역을 잘라서 분류 모델에 전달
                            detected_object = frame[y1:y2, x1:x2]
                            
                            # 분류 작업 수행
                            predicted_class = classify_image(detected_object)
                            
                            # 분류 모델이 인식된 경우 결과 표시 및 저장
                            if predicted_class is not None:
                                # 이미지에 분류 결과 표시
                                cv2.putText(frame, predicted_class, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
                                cv2.rectangle(frame, (x1, y1), (x2, y2), (36, 255, 12), 2)
                                
                                # 분류된 결과를 저장
                                # save_path = os.path.join(classified_save_dir, f"{predicted_class}_{int(time.time() * 1000)}.jpg")
                                # cv2.imwrite(save_path, detected_object)
                        except cv2.error as e:
                            print(f"OpenCV 에러: {e}")
                        except Exception as e:
                            print(f"예기치 않은 에러: {e}")
            
            # 결과 프레임을 화면에 표시
            try:
                cv2.imshow("Detection and Classification", frame)
                key = cv2.waitKey(1)
            except cv2.error as e:
                print(f"cv2.imshow() 에러: {e}")
            except Exception as e:
                print(f"cv2.waitKey() 에러: {e}")
            
        except Exception as e:
            print(f"루프 중 예외 발생: {e}")
            break  # 심각한 오류 발생 시 루프 중지

    # 모든 리소스 해제
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# 단일 스레드로 함수 실행
run_detection_and_classification()


0: 384x640 1 person, 55.9ms
Speed: 2.3ms preprocess, 55.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 224x224 rock 0.98, scissors 0.01, paper 0.00, 6.3ms
Speed: 3.3ms preprocess, 6.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 384x640 1 person, 51.7ms
Speed: 1.6ms preprocess, 51.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 224x224 rock 0.96, scissors 0.03, paper 0.01, 5.1ms
Speed: 2.3ms preprocess, 5.1ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 384x640 1 person, 49.0ms
Speed: 1.9ms preprocess, 49.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 224x224 rock 0.93, scissors 0.05, paper 0.02, 4.7ms
Speed: 1.9ms preprocess, 4.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 384x640 1 person, 52.1ms
Speed: 1.5ms preprocess, 52.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 224x224 rock 0.93, scissors 0.05, pa